In [1]:
import pandas as pd,requests,bs4,re
from datetime import datetime
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

In [2]:
SITE_NAME='Astellas'

DOMAIN = "https://www.astellas.com"

SITE_LINK="https://www.astellas.com/en/news"

In [3]:
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

In [4]:
def remove_esc_chars(text):
    return text.replace("\n", " ").replace("\t", " ").replace("\r", " ")

In [5]:
article_list = []
titles,published_dates,thumbnails,hrefs,authors,articles = [],[],[],[],[],[]
pagination = 0
last_page = 20 #250
while True:
    url = f"https://www.astellas.com/en/news?tab=latest&page={pagination}"
    soup = parse_webpage_bs(url)
    if soup != None:
        elements = soup.find_all('div',{'id' :'latest'})
        for element in elements: 
            #titles
            headlines = element.find_all('div',{'class' :'views-field-field-title'})
            for t in headlines:
                if t != None:
                    title = remove_esc_chars(t.text)
                    titles.append(title.strip())
                else:
                    titles.append("")
                    
            #dates
            dates = element.find_all('div',{'class' :'views-field-field-release-date'})
            for d in dates:
                if d != None:
                    published_date = d.text
                    published_dates.append(published_date.strip())
                else:
                    published_dates.append("")
                    
                    
            #thumbnails
            thumbnails_container = element.find_all('div',{'class' :'views-field-field-country-icon'})
            for thumb in thumbnails_container:
                if thumb != None:
                    thumbnail = thumb.find('img')
                    T = DOMAIN + thumbnail['src']
                    thumbnails.append(T.strip())
                else:
                    thumbnails.append("")
            site_all_hrefs = []
            #hrefs and authors
            hrefs_container = element.find_all('div',{'class' :'views-row'})
            for href in hrefs_container:
                if href != None:
                    href = DOMAIN + href.find('a')['href']
                    hrefs.append(href.strip())
                    site_all_hrefs.append(href.strip())
                    authors.append(SITE_NAME.strip())
                else:
                    hrefs.append("")
                    authors.append(SITE_NAME.strip())
            
            #extracting text 
            for number,link in enumerate (site_all_hrefs):
                if re.findall(".pdf",link):
                    articles.append("")
                else:
                    try:
                        soup = parse_webpage_bs(link)
                        art_containers = soup.findAll('p')
                        for art in art_containers:
                            article = remove_esc_chars(art.text)
                            articles.append(article.strip())
                    except:
                        articles.append("")
                print(f"scrapping done for url {len(site_all_hrefs) - number-1}..",end="\r")
            
    pagination += 1
    
    print(str(last_page-pagination+1) + " pages remaining..",end="\r")
    
    if pagination > last_page:
        break   

In [6]:
#zippling the list so that it'll form the tuple
zipped = list(zip(published_dates,titles,articles,thumbnails,hrefs,authors))
temp_df = pd.DataFrame(zipped,columns=['date','title','article','thumbnail','url','author'])
temp_df

,date,title,article,thumbnail,url,author
0,"May 20, 2022",Astellas Confirms Unfavorable District Court D...,Search,https://www.astellas.com/system/files/2017-01/...,https://www.astellas.com/en/news/25706,Astellas
1,"May 19, 2022",Astellas Analysis Shows No Evidence of Increas...,"The Focus Area, where we put our efforts, cons...",https://www.astellas.com/system/files/2017-01/...,https://www.astellas.com/en/news/25831,Astellas
2,"May 16, 2022",Astellas to Present New Research Further Suppo...,Astellas monitors Focus Areas from biology per...,https://www.astellas.com/system/files/2017-01/...,https://www.astellas.com/en/news/25821,Astellas
3,"May 13, 2022",Astellas Highlights Data on Advanced and Rare ...,To identify and prioritize the issues that are...,https://www.astellas.com/system/files/2017-01/...,https://www.astellas.com/en/news/25816,Astellas
4,"May 12, 2022",Notice Regarding Continuation of the Performan...,Stories about how Astellas members work to cre...,https://www.astellas.com/system/files/2017-01/...,https://www.astellas.com/system/files/news/202...,Astellas
...,...,...,...,...,...,...
100,"Mar 31, 2021",Astellas Establishes “Astellas Gene Therapies”...,Armed with the latest knowledge in medical sci...,https://www.astellas.com/system/files/2017-01/...,https://www.astellas.com/en/news/16801,Astellas
101,"Mar 31, 2021",Astellas and TB Alliance Enter into a New Coll...,We have established a global system to comply ...,https://www.astellas.com/system/files/2017-01/...,https://www.astellas.com/en/news/16796,Astellas
102,"Mar 30, 2021",Astellas’ XOSPATA® (gilteritinib) Meets Overal...,We aim to create new value and contribute to t...,https://www.astellas.com/system/files/2017-01/...,https://www.astellas.com/en/news/16781,Astellas
103,"Mar 29, 2021",Astellas Receives Positive CHMP Opinion for XT...,Search,https://www.astellas.com/system/files/2017-01/...,https://www.astellas.com/en/news/16776,Astellas


In [7]:
#to csv
temp_df.to_csv('astellas news.csv')

#to json
temp_df.to_json('astellas news.json')